# Trp-cage haMSM Construction and Analysis 
[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/jdrusso/msm_we/HEAD?labpath=examples%2Fhamsm_construction.ipynb)

For these examples, we'll be constructing an haMSM from simulations of Trp-cage unfolding.

In [1]:
from msm_we import msm_we
import numpy as np
import ray

In [2]:
# This is just to clean up the logging output for display in the documentation webpage.
msm_we.log.handlers[0]._log_render.show_time = False
msm_we.log.handlers[0].console.width = 65

## Prep

First, let's set some parameters for haMSM building.

In [3]:
h5file_paths = ['data/west.h5']

# Number of MSM microstates to initially put in each stratum/WE bin
clusters_per_stratum = 25

dimreduce_method = 'vamp'

# Boundaries of the basis/target, in progress coordinate space
pcoord_bounds = {
    'basis': [[0, 0.15]],
    'target': [[0.7, 100]]
}

model_name = 'NaCl Sample'

# Reference structure
ref_file = 'data/2JOF.pdb'

# WESTPA resampling time
tau = 1e-9

### Define MSM featurization

The function `processCoordinates` defines a transformation that's applied to the coordinates in `auxdata/coord` before dimensionality reduction/clustering.

The inputs are the coordinates (read from `auxdata/coord`) for each segment in the loaded iteration(s), and it should return an array of features for each.

We could just use full XYZ coordinates for our MSM features. 
This would be a very simple (and very poor) choice -- it doesn't capture some of the translational invariance that something like pairwise distances might.

In [4]:
def processCoordinates(self, coords):
    
    return coords.reshape(coords.shape[0], -1)
    
msm_we.modelWE.processCoordinates = processCoordinates

So, let's use pairwise distances instead, at the cost of a more expensive calculation.

Note that because of how work is serialized for parallelization, we have to define the `Universe` objects *within* `processCoordinates`. Otherwise, there would be multiple workers acting on the same object.

In [5]:
import MDAnalysis as mda
from MDAnalysis.analysis import distances

def processCoordinates(self, coords):
    
    u_ref = mda.Universe(ref_file)
    u_check = mda.Universe(ref_file)
    
    dist_out = []
    
    u_check.load_new(coords)

    for frame in u_check.trajectory:

        dists = distances.dist(
            u_check.select_atoms('backbone'),
            u_ref.select_atoms('backbone')
        )[2]

        dist_out.append(dists)

    dist_out = np.array(dist_out)
    
    return dist_out
    
msm_we.modelWE.processCoordinates = processCoordinates

## Model-building with [build_analyze_model()](../api.rst#msm_we.msm_we.modelWE.build_analyze_model)

In [6]:
model = msm_we.modelWE()

model.build_analyze_model(
    file_paths=h5file_paths,
    # ref_struct=basis_ref_dict,
    ref_struct=ref_file,
    modelName=model_name,
    basis_pcoord_bounds=pcoord_bounds['basis'],
    target_pcoord_bounds=pcoord_bounds['target'],
    dimreduce_method=dimreduce_method,
    n_clusters=clusters_per_stratum,
    tau=tau
)

Output()

2022-10-04 10:34:58,597 INFO worker.py:1509 -- Started a local 
Ray instance. View the dashboard at 127.0.0.1:8265 

Getting coordSet:   0%|          | 0/100 [00:00<?, ?it/s]

Warning: importing 'simtk.openmm' is deprecated.  Import 'openmm'
instead.

INFO     Performing weighted vamp                  ]8;id=177326;file:///home/jd/msm_we/msm_we/msm_we.py\msm_we.py]8;;\:]8;id=203319;file:///home/jd/msm_we/msm_we/msm_we.py#2786\2786]8;;\

...

Submitting fluxmatrix tasks:   0%|          | 0/49 [00:00<?, ?it/s]

Retrieving flux matrices:   0%|          | 0/49 [00:00<?, ?it/s]

INFO     Fluxmatrices all obtained                 ]8;id=758919;file:///home/jd/msm_we/msm_we/msm_we.py\msm_we.py]8;;\:]8;id=227156;file:///home/jd/msm_we/msm_we/msm_we.py#4943\4943]8;;\

INFO                                               ]8;id=567396;file:///home/jd/msm_we/msm_we/msm_we.py\msm_we.py]8;;\:]8;id=738876;file:///home/jd/msm_we/msm_we/msm_we.py#5592\5592]8;;\
         Flux converged to 3.2720e+06 after 1                    
         iterations of inverse iteration.                        

INFO     Done with steady-state estimation.        ]8;id=124360;file:///home/jd/msm_we/msm_we/msm_we.py\msm_we.py]8;;\:]8;id=808124;file:///home/jd/msm_we/msm_we/msm_we.py#5603\5603]8;;\

## Model-building step by step

[build_analyze_model()](../api.rst#msm_we.msm_we.modelWE.build_analyze_model) is just a convenient wrapper around the following steps.

You can run them manually if you want to observe each step of the process.

It's helpful to start off by running step-by-step while you're starting analysis for a system, to fine-tune parameters without re-running the entire workflow.

First, we launch Ray, which is the backend for the parallel work manager.

In [7]:
ray.init(ignore_reinit_error=True)

2022-10-04 10:35:24,406 INFO worker.py:1351 -- Calling ray.init()
again after it has already been called.

Python version:,3.9.13
Ray version:,2.0.0
Dashboard:,http://127.0.0.1:8265


In [8]:
model = msm_we.modelWE()

In [9]:
model.initialize(
    fileSpecifier=h5file_paths,
    refPDBfile=ref_file,
    modelName=model_name,
    basis_pcoord_bounds=pcoord_bounds['basis'],
    target_pcoord_bounds=pcoord_bounds['target'],
    dim_reduce_method=dimreduce_method,
    tau=tau
)

In [10]:
model.get_iterations()
model.get_coordSet(last_iter = model.maxIter, streaming=True)

Getting coordSet:   0%|          | 0/100 [00:00<?, ?it/s]

In [11]:
model.dimReduce()

INFO     Performing weighted vamp                  ]8;id=994404;file:///home/jd/msm_we/msm_we/msm_we.py\msm_we.py]8;;\:]8;id=726114;file:///home/jd/msm_we/msm_we/msm_we.py#2786\2786]8;;\

INFO     Weighted vamp will reduce 80 to 35        ]8;id=744479;file:///home/jd/msm_we/msm_we/msm_we.py\msm_we.py]8;;\:]8;id=873958;file:///home/jd/msm_we/msm_we/msm_we.py#2799\2799]8;;\
         components.                                             

`store_validation_model=True` is necessary for block validation. It enables caching the model immediately after discretization in `model.post_cluster_model`, so that the blocks can be efficiently created later.

In [12]:
model.cluster_coordinates(
    n_clusters=clusters_per_stratum,
    use_ray=True,
    stratified=True,
    store_validation_model=True # Required for block validation
)

INFO     Be aware: Number of cluster centers is an ]8;id=788003;file:///home/jd/msm_we/msm_we/msm_we.py\msm_we.py]8;;\:]8;id=358479;file:///home/jd/msm_we/msm_we/msm_we.py#3074\3074]8;;\
         important parameter, and can drastically                
         affect model quality. We recommend                      
         examining block-validation results with a               
         range of numbers of clusters, to check                  
         for overfitting.                                        

INFO     Beginning stratified clustering.          ]8;id=171943;file:///home/jd/msm_we/msm_we/msm_we.py\msm_we.py]8;;\:]8;id=486589;file:///home/jd/msm_we/msm_we/msm_we.py#3081\3081]8;;\

INFO     Obtaining bin definitions from iteration  ]8;id=623987;file:///home/jd/msm_we/msm_we/msm_we.py\msm_we.py]8;;\:]8;id=401219;file:///home/jd/msm_we/msm_we/msm_we.py#3499\3499]8;;\
         2 in file data/west.h5                                  

INFO     Loading pickled bin mapper from H5 for    ]8;id=973485;file:///home/jd/msm_we/msm_we/msm_we.py\msm_we.py]8;;\:]8;id=610701;file:///home/jd/msm_we/msm_we/msm_we.py#3503\3503]8;;\
         stratified clustering...                                

ERROR    Stratified clustering currently MUST run  ]8;id=72796;file:///home/jd/msm_we/msm_we/msm_we.py\msm_we.py]8;;\:]8;id=484150;file:///home/jd/msm_we/msm_we/msm_we.py#3539\3539]8;;\
         in streaming mode (and with ray enabled                 
         for discretization). Enabling both and                  
         continuing...                                           

...

Clustering:   0%|          | 0/99 [00:00<?, ?it/s]

INFO     Using Ray cluster with 16.0 CPUs!          ]8;id=124766;file:///home/jd/msm_we/msm_we/msm_we.py\msm_we.py]8;;\:]8;id=9387;file:///home/jd/msm_we/msm_we/msm_we.py#873\873]8;;\

Submitting discretization tasks:   0%|          | 0/99 [00:00<?, ?it/s]

Retrieving discretized trajectories:   0%|          | 0/99 [00:00<?, ?it/s]

INFO     Discretization complete                   ]8;id=61497;file:///home/jd/msm_we/msm_we/msm_we.py\msm_we.py]8;;\:]8;id=124131;file:///home/jd/msm_we/msm_we/msm_we.py#4194\4194]8;;\

In [13]:
model.get_fluxMatrix(n_lag=0)

Constructing flux matrix:   0%|          | 0/98 [00:00<?, ?it/s]

It's likely our flux matrix isn't strongly connected. This cleaning step removes all clusters that aren't in the largest connected set, then rediscretizes all the trajectories according to the new, reduced set of clusters.

In [14]:
model.organize_fluxMatrix()

INFO     Started with 300 clusters, and removed    ]8;id=167239;file:///home/jd/msm_we/msm_we/msm_we.py\msm_we.py]8;;\:]8;id=170712;file:///home/jd/msm_we/msm_we/msm_we.py#4018\4018]8;;\
         133                                                     

WARNING  All clusters were cleaned from bins {11}  ]8;id=296936;file:///home/jd/msm_we/msm_we/msm_we.py\msm_we.py]8;;\:]8;id=331241;file:///home/jd/msm_we/msm_we/msm_we.py#4034\4034]8;;\

INFO     Using Ray cluster with 16.0 CPUs!          ]8;id=27036;file:///home/jd/msm_we/msm_we/msm_we.py\msm_we.py]8;;\:]8;id=595664;file:///home/jd/msm_we/msm_we/msm_we.py#873\873]8;;\

INFO     Using cached model for discretization     ]8;id=960711;file:///home/jd/msm_we/msm_we/msm_we.py\msm_we.py]8;;\:]8;id=156149;file:///home/jd/msm_we/msm_we/msm_we.py#4130\4130]8;;\

Submitting discretization tasks:   0%|          | 0/99 [00:00<?, ?it/s]

Retrieving discretized trajectories:   0%|          | 0/99 [00:00<?, ?it/s]

INFO     Discretization complete                   ]8;id=600521;file:///home/jd/msm_we/msm_we/msm_we.py\msm_we.py]8;;\:]8;id=633284;file:///home/jd/msm_we/msm_we/msm_we.py#4194\4194]8;;\

Constructing flux matrix:   0%|          | 0/98 [00:00<?, ?it/s]

### Transition matrix estimation

In [15]:
model.get_Tmatrix()

In [16]:
model.get_steady_state()

INFO                                               ]8;id=182667;file:///home/jd/msm_we/msm_we/msm_we.py\msm_we.py]8;;\:]8;id=400843;file:///home/jd/msm_we/msm_we/msm_we.py#5592\5592]8;;\
         Flux converged to 2.1088e+06 after 1                    
         iterations of inverse iteration.                        

INFO     Done with steady-state estimation.        ]8;id=377393;file:///home/jd/msm_we/msm_we/msm_we.py\msm_we.py]8;;\:]8;id=726915;file:///home/jd/msm_we/msm_we/msm_we.py#5603\5603]8;;\

In [17]:
model.get_steady_state_target_flux()

### Model validation

This block validation step 

In [18]:
model.do_block_validation(
    cross_validation_groups=2, 
    cross_validation_blocks=4
)

INFO     Beginning analysis of cross-validation    ]8;id=996098;file:///home/jd/msm_we/msm_we/msm_we.py\msm_we.py]8;;\:]8;id=908382;file:///home/jd/msm_we/msm_we/msm_we.py#1339\1339]8;;\
         group 1/2.                                              

INFO     Using Ray cluster with 16.0 CPUs!          ]8;id=266577;file:///home/jd/msm_we/msm_we/msm_we.py\msm_we.py]8;;\:]8;id=82631;file:///home/jd/msm_we/msm_we/msm_we.py#873\873]8;;\

Submitting fluxmatrix tasks:   0%|          | 0/50 [00:00<?, ?it/s]

Retrieving flux matrices:   0%|          | 0/50 [00:00<?, ?it/s]

INFO     Fluxmatrices all obtained                 ]8;id=705609;file:///home/jd/msm_we/msm_we/msm_we.py\msm_we.py]8;;\:]8;id=846566;file:///home/jd/msm_we/msm_we/msm_we.py#4943\4943]8;;\

...

Submitting fluxmatrix tasks:   0%|          | 0/49 [00:00<?, ?it/s]

Retrieving flux matrices:   0%|          | 0/49 [00:00<?, ?it/s]

INFO     Fluxmatrices all obtained                 ]8;id=623637;file:///home/jd/msm_we/msm_we/msm_we.py\msm_we.py]8;;\:]8;id=222446;file:///home/jd/msm_we/msm_we/msm_we.py#4943\4943]8;;\

INFO                                               ]8;id=189340;file:///home/jd/msm_we/msm_we/msm_we.py\msm_we.py]8;;\:]8;id=144328;file:///home/jd/msm_we/msm_we/msm_we.py#5592\5592]8;;\
         Flux converged to 3.2720e+06 after 1                    
         iterations of inverse iteration.                        

INFO     Done with steady-state estimation.        ]8;id=73221;file:///home/jd/msm_we/msm_we/msm_we.py\msm_we.py]8;;\:]8;id=93556;file:///home/jd/msm_we/msm_we/msm_we.py#5603\5603]8;;\

## Save model

In [ ]:
import pickle
with open('data/pickled_model', 'wb') as of:
    pickle.dump(model, of)